In [3]:
from sqlalchemy import create_engine
import datetime as dt
import pandas as pd
import getpass

db_user = getpass.getpass('user')
db_password = getpass.getpass('password')

redshift_engine = create_engine('postgresql+psycopg2://{}:{}@bun-analytics.chlb9dm0sjmd.ap-northeast-2.redshift.amazonaws.com:5439/bunjang'
                                .format(db_user, db_password))
redshift = redshift_engine.connect()

user ·········
password ········


ModuleNotFoundError: No module named 'psycopg2'

In [2]:
sqls = [
'''
drop table if exists workspace.tmp_search_keyword_step1
''',
'''
create table workspace.tmp_search_keyword_step1 as
select
distinct
content_id as pid,
user_id as uid,
ref_term as keyword
from bun_log_app.app_event_type_impression
where year||month||day||hour >= '2020012103' and year||month||day||hour < '2020012803'
    and ref_term is not null
    and ref_campaign is not null
''',
'''
drop table if exists workspace.tmp_search_keyword_step2
''',
'''
create table workspace.tmp_search_keyword_step2 as
select
    ue.age as generation,
    ue.sex,
    left(c.category, 3) as category1,
    --c.name as category_name,
    sp.keyword,
    count(*) cnt
from workspace.tmp_search_keyword_step1 sp
    join service1_quicket.user_ u
        on sp.uid = u.id
    join service1_quicket.product_info p
        on sp.pid = p.id
    join service1_quicket.categories c
        on c.category = p.category_id
    left join service1_quicket.user_extra_info ue on u.id = ue.uid
where
    ue.sex != 0
    and ue.age != 0
    and left(c.category, 3) in (
        '320', '700', '600', '400', '750', '200', '310', '910', '900', '800', '410', '500'
    )
group by
    1, 2, 3, 4
''',
'''
drop table if exists workspace.tmp_search_keyword_step3
''',
'''
create table workspace.tmp_search_keyword_step3 as
select *
from (
    select
        *,
        row_number() over (partition by generation, sex, category1 order by cnt desc) as rank
    from workspace.tmp_search_keyword_step2
)
where rank < 25
order by generation, sex, category1, rank
'''    
]

for sql in sqls:
    redshift.execute(sql)


In [3]:
sql = '''
select
    *
from workspace.tmp_search_keyword_step3
'''
keyword_stat = pd.read_sql(sql, redshift)
keyword_stat.head()

,generation,sex,category1,keyword,cnt,rank
0,10,1,200,무료나눔,4393,1
1,10,1,200,무나,512,3
2,10,1,200,지갑,168,5
3,10,1,200,명품개꿀매,114,7
4,10,1,200,무료 나눔,104,9


In [4]:
sqls = [
'''
drop table if exists workspace.tmp_search_keyword_step4
''',
'''
create table workspace.tmp_search_keyword_step4 as
select
    left(c.category_id, 3) as category1,
    d.keyword,
    count(*) as cnt
from (
    select distinct suid
    from service1_quicket.ad_super_up_point
    where create_at >= '2020-01-21 00:00:00' and create_at <= '2020-01-27 23:59:59'
) a
join service1_quicket.ad_super_up b
    on a.suid = b.suid
join service1_quicket.product_info c
    on b.pid = c.id
join service1_quicket.ad_super_up_basic_keyword d
    on c.name like '%%'||d.keyword||'%%'
group by 1, 2
'''
]

for sql in sqls:
    redshift.execute(sql)

sql = '''
    select * from workspace.tmp_search_keyword_step4
'''

category_keyword_count = pd.read_sql(sql, redshift)
category_keyword_count.head()

,category1,keyword,cnt
0,600,노트5,4
1,400,고야드,8
2,400,남녀공용,11
3,400,보테가,19
4,900,후드,1


In [5]:
sql = '''
select
    x.*,
    y.cnt as category_count
from (
    select
        generation, sex, category1, a.keyword, a.cnt, a.rank, b.count as ad_count, b.price as ad_price
    from workspace.tmp_search_keyword_step3 a
    left join service1_quicket.ad_super_up_basic_keyword b on a.keyword = b.keyword
) x
left join workspace.tmp_search_keyword_step4 y on x.keyword = y.keyword and x.category1 = y.category1
'''

df = pd.read_sql(sql, redshift)
df.head()

,generation,sex,category1,keyword,cnt,rank,ad_count,ad_price,category_count
0,10,1,200,나눔,795,2,3.0,2800.0,24.0
1,10,1,200,숏패딩,291,4,39.0,2940.0,NaN
2,10,1,200,무스너클 맨투맨,122,6,NaN,NaN,NaN
3,10,1,200,맨투맨,110,8,110.0,3080.0,8.0
4,10,1,200,입생로랑 지갑,99,10,NaN,NaN,NaN


In [6]:
df.drop_duplicates().to_excel('category_keyword.xlsx')
df.drop_duplicates().to_csv('category_keyword.csv')